In [1]:
import weaviate, os

In [2]:
!pip install -U weaviate-client


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
AWS_BEDROCK_ACCESS_KEY_ID=''
AWS_ACCESS_KEY_ID=''

AWS_BEDROCK_SECRET_ACCESS_KEY=''
AWS_SECRET_ACCESS_KEY=''


AWS_BEDROCK_REGION='us-east-1'
AWS_BEDROCK_SESSION_TOKEN=''
AWS_SESSION_TOKEN=''

BEDROCK_ENDPOINT='https://bedrock.amazonaws.com'
MODEL_ID='cohere.embed-english-v3'
REGION_NAME='us-east-1'

auth_credentials=''

cluster_url=''


In [28]:
weaviate.__version__

'4.6.5'

In [4]:
import weaviate
import os
import requests
import json
import weaviate.classes.config as wc

# AWS Credentials
AWS_ACCESS_KEY_ID = os.getenv('AWS_BEDROCK_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_BEDROCK_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_BEDROCK_SESSION_TOKEN')

# Bedrock Endpoint
BEDROCK_ENDPOINT = 'https://bedrock.amazonaws.com'

# Weaviate Connection

client = weaviate.connect_to_wcs(
     cluster_url=cluster_url,                             # Replace with your WCS cluster ID
     auth_credentials=weaviate.auth.AuthApiKey(auth_credentials), # Replace with your WCS API KEY - recommended: use env var
      headers={
          "X-AWS-Access-Key": AWS_BEDROCK_ACCESS_KEY_ID, # Replace with your AWS access key - recommended: use env var
          "X-AWS-Secret-Key": AWS_BEDROCK_SECRET_ACCESS_KEY, # Replace with your AWS secret key - recommended: use env var
          "X-AWS-Session-Token": AWS_BEDROCK_SESSION_TOKEN

        },
    #timeout_config=weaviate.util.Config.Timeout(init=60),  # Increase init timeout to 60 seconds
    #startup_config=weaviate.util.Config.Startup(skip_init_checks=True)  # Skip startup checks
)

print(f'Clinet Ready  with connect : {client.is_ready()}')

Clinet Ready  with connect : True


In [6]:
# Set the folder path containing the documents
folder_path = './output_files/'

In [50]:
# Delete the collection if it already exists
if (client.collections.exists("PersonProfile")):
    client.collections.delete("PersonProfile")

client.collections.create(
    name="PersonProfile",
    

    vectorizer_config=wc.Configure.Vectorizer.text2vec_aws(
        model="cohere.embed-english-v3", # select the model, make sure it is enabled for your account
        service="bedrock",
        #source_properties=["title"],
        region="us-east-1"               # select your region
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="content", data_type=wc.DataType.TEXT), 
        
    ]
)

In [53]:
# Initialize an empty list to store the data
data = []

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a text document
    if filename.endswith(".txt"):
        # Read the document
        with open(os.path.join(folder_path, filename), 'r') as file:
            document_text = file.read()
            data.append(document_text)

# Now data contains the text content of all the files
print(f"Read {len(data)} text files.")


# collection = client.collections.get("PersonProfile")

# # Loop through each file in the folder
# for filename in os.listdir(folder_path):
#   # Check if the file is a text document
#   if filename.endswith(".txt"):
    
#     # Read the document
#     with open(os.path.join(folder_path, filename), 'r') as file:
#       document_text= file.read()      
  

Read 999 text files.


In [56]:

collection = client.collections.get("PersonProfile")
    
with collection.batch.dynamic() as batch:
    for src_obj in data:
        weaviate_obj = {
            "content": src_obj
        }

        # The model provider integration will automatically vectorize the object
        batch.add_object(
            properties=weaviate_obj,
            
        )

In [57]:
coll = client.collections.get("PersonProfile")
response = coll.query.fetch_objects()

for o in response.objects:
    print(o.properties)

{'content': 'Education Summary:\nJavier Martinez studied Economics;Human Resources;Business Administration at Dixie State University and graduated in 2009-01-01.\nAggregate Summary:\nJavier Martinez has over 22.6 years of experience across the following job titles: Master Scheduler, Metrology Technician, Quality Assurance Technician, Enclosures / E-Coat Supervisor, Area Manager, Operations Manager, Production Trainer, Material Planning and Logistics Supervisor, Stamping Supervisor, Body Shop - Super Duty, Supervisor, Management Consultant, Operations Manager at companies: Deseret Laboratories, Deseret Laboratories, Deseret Laboratories, Tesla, Amazon, Amazon, Viracon, Ford Motor, Ford Motor, Ford Motor, Advantus Global Inc, FedEx with an average revenue size of $34,489,074,733.15.\n--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n\n'}


In [59]:
from weaviate.classes.query import MetadataQuery

coll = client.collections.get("PersonProfile")
response = coll.query.fetch_objects()

#print(response)

response = coll.query.near_text(
    query="person worked or working in apple",
    limit=10,
    return_metadata=MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)

{'content': 'Employment Summary:\nMike Crowe is currently working as a Director Creative Director iPhone at Apple,  with 224704 employees and a revenue of $31,949,031,445.00. \nEducation Summary:\nMike Crowe studied Design at Curtin University and graduated in 1996-01-01.\nAggregate Summary:\nMike Crowe has over 34.0 years of experience across the following job titles: Creative, Creative Director iPhone, Creative Partner, Creative Director, Creative Director at companies: Ogilvy, Apple, AMV BBDO, DDB London, adam&eveDDB with an average revenue size of $5,324,838,574.17.\n--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n\n'}
0.4804850220680237
{'content': 'Employment Summary:\nRichardManoj  ShustermanPM has worked as a Head of ... Head Of Foundation, Apple Online Store Engineering at Apple,  with 224704 employees and a revenue of $31,94

In [60]:
from weaviate.classes.query import MetadataQuery

coll = client.collections.get("PersonProfile")
response = coll.query.fetch_objects()

#print(response)

response = coll.query.near_text(
    query="person worked any company revenue more than 100 millions",
    limit=10,
    return_metadata=MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)

{'content': 'Employment Summary:\nNitin Singhal has worked as a Manager Engineering Manager - Data Platform and Engineering at PayPal,  with 29987 employees and a revenue of $4,246,278,479.00.  Nitin Singhal has worked as a Manager Engineering Manager at PayPal,  with 29987 employees and a revenue of $4,246,278,479.00.  Nitin Singhal has worked as a Director Director, Enterprise Data Management, and Privacy Platforms at Twitter,  with 3346 employees and a revenue of $1,411,687,950.00.  Nitin Singhal has worked as a Director Director, Data Management at Twitter,  with 3346 employees and a revenue of $1,411,687,950.00.  Nitin Singhal has worked as a Director Sr. Director, Engineering at Twitter,  with 3346 employees and a revenue of $1,411,687,950.00.  Nitin Singhal has worked as a Director Sr. Director - Head of Data products at Visa,  with 24541 employees and a revenue of $5,413,728,495.00. \nEducation Summary:\nNitin Singhal studied Engineering at Delhi College Of Engineering and grad